### Athenticate kfp SDK with Pipeline API

Follow: https://www.kubeflow.org/docs/aws/pipeline/#authenticate-kubeflow-pipeline-using-sdk-inside-cluster
Get the session cookie from your browser and replace Content of cookie with <cookie>
Get ALB address and replace with <aws_alb_host>. Also modify namespace with <your_namespace>

In [ ]:
import kfp
authservice_session='authservice_session=<cookie>'
client = kfp.Client(host='http://<aws_alb_host>/pipeline', cookies=authservice_session)
client.list_experiments(namespace="<your_namespace>")

In [ ]:
import kfp.dsl as dsl
from kfp import compiler

#from irml_tim.kubeflow import transformers
from kubernetes import client as k8s_client
from kubernetes.client.models import V1EnvVar, V1SecretKeySelector

In [ ]:
def node_selector(op):
    if isinstance(op, dsl.ContainerOp):
        # op.add_node_selector_constraint('compute-size', 'cpu-small')
        # op.add_node_selector_constraint('single-az', 'true')
        op.add_node_selector_constraint('spot', 'false')
        op.container.set_memory_request("2G")
        op.container.set_cpu_request("1")

In [ ]:
@dsl.pipeline(
    name="VolumeOp Basic",
    description="A Basic Example on VolumeOp Usage."
)
def ebs_pipeline(size="1Gi"):

    vop = dsl.VolumeOp(
        name="create_pvc",
        resource_name="my-pvc",
        modes=dsl.VOLUME_MODE_RWO,
        size=size
    )

    cop = dsl.ContainerOp(
        name="Component1",
        image="library/bash:4.4.23",
        command=["sh", "-c"],
        arguments=["sleep 1m && echo foo > /mnt/file1"],
        pvolumes={"/mnt": vop.volume}
    )

    cop2 = dsl.ContainerOp(
        name="Component2",
        image="library/bash:4.4.23",
        command=["sh", "-c"],
        arguments=["cat /mnt/file1"],
        pvolumes={"/mnt": vop.volume}
    ).after(cop)
    
    vop.delete().after(cop2)

    pipeline_conf = dsl.get_pipeline_conf()
#     pipeline_conf.add_op_transformer(transformers.irml_defaults)
#     pipeline_conf.add_op_transformer(node_selector)


sample_input = {'size': '1Gi'}    

In [ ]:
# Get or create an experiment and submit a pipeline run
EXPERIMENT_NAME='ebs-delete'
experiment = client.create_experiment(name=EXPERIMENT_NAME, namespace='eksworkshop')

In [ ]:
pipeline_func = ebs_pipeline
pipeline_filename = pipeline_func.__name__ + '.zip'
compiler.Compiler().compile(pipeline_func, pipeline_filename)

In [ ]:
# Specify pipeline argument values
arguments = {'size': '1Gi'}

# Submit a pipeline run
run_name = pipeline_func.__name__ + ' run'
run_result = client.run_pipeline(experiment.id, run_name, pipeline_filename, arguments)

# This link leads to the run information page. 
# Note: There is a bug in JupyterLab that modifies the URL and makes the link stop working